In [1]:
#pragma cling add_include_path("/home/german/chemfiles/build/install/include")
#pragma cling add_library_path("/home/german/chemfiles/build/install/lib")
#include "chemfiles.hpp"
#pragma cling load("libchemfiles.so")
#include <iostream>
#include <string>
#include <typeinfo>

In [2]:
// Read PDB and get its topology
chemfiles::Trajectory in_top_trj("/home/german/labo/18/lbp/run/4xcp/pca/full_avg_4xcp.pdb");
auto in_top_frm = in_top_trj.read();
auto in_top = in_top_frm.topology();
// Get ligand topology
auto residuos = in_top.residues();
auto lig_res = residuos[residuos.size() - 1];
// Add it to the output topology
chemfiles::Topology out_top;
for (const auto &each : lig_res) {
    out_top.add_atom(in_top[each]);
}

In [3]:
chemfiles::Trajectory in_trj("/home/german/labo/18/lbp/run/4xcp/data/full_fit_4xcp.nc");
std::string out_folder = "/home/german/labo/18/lbp/run/4xcp/lig_asa/pdbs/";
std::string out_filename = "traj_";

In [4]:
for (size_t i = 0 ; i < in_trj.nsteps() ; i+= 10) {
    auto in_frm = in_trj.read_step(i);
    auto in_xyz = in_frm.positions();
    
    chemfiles::Frame out_frm;

    size_t j = 0;
    auto res_ite = std::begin(lig_res);
    for(const auto& lig_atm : lig_res) {
        out_frm.add_atom(chemfiles::Atom(in_top[lig_atm].type()), in_xyz[*res_ite++]);
        ++j;
    }
    chemfiles::Trajectory out_trj(out_folder + out_filename + std::to_string(i) + ".pdb", 'w');
    out_trj.set_topology(out_top);   
    out_trj.write(out_frm);
}